# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Adding the CSV
csv_path = "C:/Users/prakb/OneDrive/Desktop/python-api-challenge/output_data/cities.csv"
# Reading the CSV
cities_csv = pd.read_csv(weather_data_path)
cities_csv.head()


,Unnamed: 0,City,Clouds,Dates,Humidity,Latitude,Longitude,Temperature,Wind Speed,Country
0,0,Khanpur,27,1659584459,40,28.6453,70.6567,97.70,9.44,PK
1,1,Rikitea,97,1659584459,72,-23.1203,-134.9692,71.38,20.36,PF
2,2,Atuona,36,1659584459,81,-9.8000,-139.0333,77.58,19.84,PF
3,3,Bathsheba,75,1659584460,89,13.2167,-59.5167,82.06,19.57,BB
4,4,Jamestown,100,1659584455,73,42.0970,-79.2353,76.64,0.00,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure Gmaps
gmaps.configure(api_key=g_key)

In [ ]:
Humidity = weather_csv["Humidity"].astype(float)

In [12]:
#Humidity Heatmap
Heatmap1 = cities_csv[["Latitude", "Longitude"]].astype(float)
fig = gmaps.figure()

# Creating the Heat Layer
Heat_Layer = gmaps.heatmap_layer(Heatmap1, weights=Humidity, max_intensity=100, dissipating=False, point_radius=4)

fig.add_layer(Heat_Layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# Hot Desert Locations
Cities_new = cities_csv.loc[cities_csv["Humidity"]>20]
Cities_new = Cities_new.loc[cities_csv["Temperature"]>90]
Cities_new = Cities_new.loc[cities_csv["Wind Speed"]<10]
Cities_new = Cities_new.loc[cities_csv["Clouds"]<10]
Cities_new

,Unnamed: 0,City,Clouds,Dates,Humidity,Latitude,Longitude,Temperature,Wind Speed,Country
160,160,Huoqiu,1,1659584507,49,32.3394,116.2694,96.49,5.08,CN
162,162,Borazjan,6,1659584507,34,29.2666,51.2159,101.71,0.78,IR
259,259,Saint George,0,1659584537,24,37.1041,-113.5841,97.39,0.00,US
270,270,Danjiangkou,1,1659584540,48,32.5428,111.5086,98.20,4.36,CN
413,413,Sharjah city,0,1659584583,66,25.3573,55.4033,93.49,9.22,AE
440,440,Kaifeng,0,1659584592,44,34.7911,114.3483,98.42,8.61,CN
490,490,Yuba City,0,1659584471,31,39.1405,-121.6169,96.78,6.91,US


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
#hotel Dataframe for Desert Locations
hotel_df = Cities_new[["City", "Latitude", "Longitude", "Country"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Latitude"] = ""
hotel_df["Hotel Longitude"] = ""

hotel_df.head()

,City,Latitude,Longitude,Country,Hotel Name,Hotel Latitude,Hotel Longitude
160,Huoqiu,32.3394,116.2694,CN,,,
162,Borazjan,29.2666,51.2159,IR,,,
259,Saint George,37.1041,-113.5841,US,,,
270,Danjiangkou,32.5428,111.5086,CN,,,
413,Sharjah city,25.3573,55.4033,AE,,,


In [56]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    City_location = row["City"]
    Latitude = row["Latitude"]
    Longitude = row["Longitude"]
    query_url = f'{url}?key={g_key}&location={Latitude},{Longitude}&radius=5000&type=lodging'
    response = requests.get(query_url).json()
    
    try:
        print(f"Nearby Hotels in {City_location}.")
        hotel_name = response["results"][0]['name']
        print(f"Hotel Name: {hotel_name}")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        hotel_df.loc[index, 'Hotel Latitude'] = response["results"][0]['geometry']['location']['Latitude']
        hotel_df.loc[index, 'Hotel Longitude'] = response["results"][0]['geometry']['location']['Longitude']
    except:
        pass

Nearby Hotels in Huoqiu.
Hotel Name: GreenTree Inn AnHui LuAn HuoQiu JianXing Road Express Hotel
Nearby Hotels in Borazjan.
Hotel Name: شرکت زرمهریار شعبه سوم
Nearby Hotels in Saint George.
Hotel Name: Best Western Plus Abbey Inn
Nearby Hotels in Danjiangkou.
Hotel Name: Dongsheng Guesthouse
Nearby Hotels in Sharjah city.
Hotel Name: Swiss Belhotel Sharjah
Nearby Hotels in Kaifeng.
Hotel Name: Pullman Kaifeng Jianye
Nearby Hotels in Yuba City.
Hotel Name: Bonanza Inn & Suites Yuba City


In [50]:
hotel_df

,City,Latitude,Longitude,Country,Hotel Name,Hotel Latitude,Hotel Longitude
160,Huoqiu,32.3394,116.2694,CN,GreenTree Inn AnHui LuAn HuoQiu JianXing Road ...,,
162,Borazjan,29.2666,51.2159,IR,شرکت زرمهریار شعبه سوم,,
259,Saint George,37.1041,-113.5841,US,Best Western Plus Abbey Inn,,
270,Danjiangkou,32.5428,111.5086,CN,Dongsheng Guesthouse,,
413,Sharjah city,25.3573,55.4033,AE,Swiss Belhotel Sharjah,,
440,Kaifeng,34.7911,114.3483,CN,Pullman Kaifeng Jianye,,
490,Yuba City,39.1405,-121.6169,US,Bonanza Inn & Suites Yuba City,,


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [55]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))